EDAS is an abbreviation for **Evaluation Based on Distance** from the Average Solution. It is one of the methods in multi-criteria decision making.

# PREPARATION

In [1]:
import math
from itertools import permutations
from itertools import product
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display
import json

## Global Helper Function

In [2]:
def create_criteria_decision_form() -> None: 
    global total_criteria
    
    dropdowns = []
    for i in range(total_criteria):
        label = widgets.Label(value=f"C{i+1}", layout=widgets.Layout(width="50px"))
    
        dropdown = widgets.Dropdown(options=['Benefit', 'Cost'], layout=widgets.Layout(width="100px"))
        dropdowns.append(dropdown)
    
        display(widgets.HBox([label, dropdown]))

    def get_dropdown_values():
        return [dropdown.value for dropdown in dropdowns]
    
    output = widgets.Output()
    
    def set_criteria(_):
        global criterias
        
        criterias['type'] = get_dropdown_values()
            
        with output:
            output.clear_output(wait=True)
            print('Criterias')
            print(criterias)
        
    set_criteria_btn = widgets.Button(description="Set Criteria!")
    set_criteria_btn.on_click(set_criteria)

    display(set_criteria_btn, output)

---

# Input Matrix-Like Data Structure of Criterias and Alternatives

In [7]:
total_criteria = 5
total_alternative = 5

criteria_labels = tuple(f"C{i + 1}" for i in range(total_criteria))
alternative_labels = tuple(f"A{i + 1}" for i in range(total_alternative))

criterias = pd.DataFrame(None, index=criteria_labels, columns=['type'])

In [6]:
create_criteria_decision_form()

Button(description='Set Criteria!', style=ButtonStyle())

Output()

# Calculate Average Solution (AV)

In [ ]:
print("Calculation should be here")

# Calculate Positive Distance Criteria (PDA) and Negative Distance Criteria (NDA)

In [ ]:
print("Calculation should be here")

# Calculate Sum Weight of PDA and NDA (SP and SN)

In [ ]:
print("Calculation should be here")

# Normalization SP and SN

In [ ]:
print("Calculation should be here")

# Calculate Aprasial Score (AS)

In [ ]:
print("Calculation should be here")

# Rangking The Alternatives

In [ ]:
print("Calculation should be here")